# PyTorch 张量

张量是一种专门化的数据结构，与数组和矩阵非常相似。在 PyTorch 中，使用张量来编码模型的输入和输出，以及模型的参数。

张量类似于 [NumPy](https://numpy.org/) 的 ndarray，不同之处在于张量可以在 GPU 或其他硬件加速器上运行。事实上，张量和 NumPy 数组通常可以共享相同的底层内存，这样就无需复制数据（参见 [](bridge-to-np-label)）。张量还针对自动微分进行了优化（将在后面的[Autograd](autogradqs)部分更详细地讨论这一点）。如果你对 ndarray 很熟悉，那么你会对 Tensor API 感到如鱼得水。如果不是，请继续跟随！

In [1]:
import torch
import numpy as np

## 初始化张量

张量可以通过多种方式进行初始化。请看以下示例：

### 直接从数据创建

张量可以直接从数据中创建，数据类型会自动推断。

In [2]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

### 从 NumPy 数组创建张量

可以从 NumPy 数组创建张量（反之亦然，参见 [](bridge-to-np-label)）。

In [3]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

### 从另一个张量中

新张量保留参数张量的属性（形状、数据类型），除非明确覆盖。

In [4]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.2507, 0.7931],
        [0.9123, 0.4217]]) 



### 随机或常数值

`shape` 是张量维度的元组。在以下函数中，它决定了输出张量的维数。

In [5]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.1300, 0.3569, 0.9689],
        [0.8137, 0.9024, 0.8713]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## 张量的属性

张量的属性描述了它们的形状、数据类型以及它们存储的设备。

In [6]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## 张量运算

超过 100 种[张量运算]((https://pytorch.org/docs/stable/torch.html))，包括算术运算、线性代数、矩阵操作（转置、索引、切片）、采样等。

这些运算都可以在 GPU 上运行（通常比在 CPU 上速度快）。如果你使用的是 Colab，可以通过“运行时”> “更改运行时类型” > “GPU”来分配 GPU。

默认情况下，张量是在 CPU 上创建的。需要使用 `.to` 方法明确地将张量移动到 GPU（在检查 GPU 可用性之后）。请记住，跨设备复制大张量可能会在时间和内存方面非常昂贵！

In [7]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

尝试列表中的一些运算。如果你对 NumPy API 有所了解，你会发现 Tensor API 非常容易使用。
### 标准的类似 numpy 的索引和切片

In [8]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


### 合并张量

你可以使用 {func}`torch.cat` 来沿着指定的维度拼接一系列张量。同时请参阅 {class}`torch.stack`，这是另一个与 {func}`torch.cat` 略有不同的张量拼接算子。

In [9]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


### 算术运算

此代码用于计算两个张量之间的矩阵乘法。`y1`、`y2`、`y3` 的值将相同：

In [10]:
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

### 单元素张量 

如果你有单元素的张量，例如，通过将所有张量的值聚合成一个值，你可以使用 `item()` 方法将其转换为 Python 的数值型数据。

In [11]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


### 就地更改

将结果存储到操作数（operand）中称为就地更改（in-place）。它们通过 `_` 后缀来表示。例如：`x.copy_(y)`, `x.t_()`，会改变 `x` 的值。

In [12]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


```{note}
In-place 虽然能够节省一些内存，但在计算导数时可能会因为立即失去历史信息而出现问题。因此，不建议使用这些运算。
```

## 复制张量

与 Python 中的任何对象一样，将张量分配给变量会使变量成为张量的 **标签**，而不会复制它。例如

In [13]:
a = torch.ones(2, 2)
b = a

a[0][1] = 561  # we change a...
print(b)       # ...and b is also altered

tensor([[  1., 561.],
        [  1.,   1.]])


但是，如果想要单独的数据副本以进行操作呢？`clone()` 方法可以帮助您。

In [14]:
a = torch.ones(2, 2)
b = a.clone()

assert b is not a      # different objects in memory...
print(torch.eq(a, b))  # ...but still with the same contents!

a[0][1] = 561          # a changes...
print(b)               # ...but b is still all ones

tensor([[True, True],
        [True, True]])
tensor([[1., 1.],
        [1., 1.]])


**使用 ``clone()`` 时需要注意重要的一点。** 如果您的源张量启用了自动梯度，那么克隆也会启用。**这将在自动梯度视频中更深入地介绍，**但是，如果您想要了解详细信息的简化版本，请继续阅读。

在许多情况下，这将是您想要的。例如，如果您的模型在其 `forward()` 方法中有多个计算路径，并且 **原始张量和它的克隆** 都对模型的输出有贡献，那么为了启用模型学习，您需要为这两个张量打开自动梯度。如果您的源张量启用了自动梯度（如果它是一组学习权重或源自涉及权重的计算，它通常会启用），那么您将获得想要的结果。

另一方面，如果您正在进行计算，其中 **原始张量和它的克隆** 都不需要跟踪梯度，那么只要源张量关闭了自动梯度，您就可以放心地进行计算。

还有第三种情况：假设您在模型的 `forward()` 函数中执行计算，默认情况下所有内容都打开了梯度，但您希望在中途提取一些值以生成一些指标。在这种情况下，您 **不** 希望源张量的克隆副本跟踪梯度 - 关闭自动梯度的历史跟踪可以提高性能。为此，您可以对源张量使用. `detach()` 方法

In [15]:
a = torch.rand(2, 2, requires_grad=True) # turn on autograd
print(a)

b = a.clone()
print(b)

c = a.detach().clone()
print(c)

print(a)

tensor([[0.1441, 0.8971],
        [0.7579, 0.5853]], requires_grad=True)
tensor([[0.1441, 0.8971],
        [0.7579, 0.5853]], grad_fn=<CloneBackward0>)
tensor([[0.1441, 0.8971],
        [0.7579, 0.5853]])
tensor([[0.1441, 0.8971],
        [0.7579, 0.5853]], requires_grad=True)


这里发生了什么？

- 创建了 `a`，其中 `requires_grad=True` 已启用。
- 当我们打印 `a` 时，它会告诉我们 `requires_grad=True` 属性 - 这意味着自动梯度和计算历史跟踪已启用。
- 克隆了 `a` 并将其标记为 `b`。当我们打印b时，可以看到它正在跟踪其计算历史 - 它继承了 `a` 的自动梯度设置，并添加到计算历史中。
- 克隆了 `a` 到 `c` 中，但先调用了 `detach()`。
- 打印 `c`，没有看到计算历史，也没有 `requires_grad=True`。

`detach()` 方法 **将张量与其计算历史分离**。它表示：“将接下来的任何操作视为自动梯度已关闭一样执行。”它 **不会** 更改 `a` - 您可以在最后打印 `a` 时看到，它保留了 `requires_grad=True` 属性。

(bridge-to-np-label)=
## 桥接 NumPy

CPU 上的张量和 NumPy 数组可以共享其底层内存位置，改变其中一个将会影响另一个。

### 张量转 NumPy 数组

In [16]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


在 NumPy 数组中反映出张量的变化。

In [17]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### NumPy 数组转张量

In [18]:
n = np.ones(5)
t = torch.from_numpy(n)

NumPy 数组的变更反映在张量上。

In [19]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
